# 推荐系统测评
## 实验方法
- 离线实验
- 用户调查
- 在线实验

## 测评指标
- 用户满意度
- 预测准确度
    + 评分预测
    + TopN推荐
- 覆盖率
- 多样性
- 相似性
- 新颖性
- 惊喜度
- 信任度
- 实时性
- 健壮性

## 维度测评
- 用户维度
- 物品维度
- 时间维度


# 第2章 利用用户行为数据

- 无上下文信息的隐性反馈数据集：每一条记录仅仅包含用户ID和物品ID

- 无上下文信息的显性反馈数据集：每一条记录包含用户ID、物品ID和用户对物品的评分

- 有上下文信息的隐性反馈数据集：每一条记录包含用户ID、物品ID和用户对物品的评分和评分行为发生的时间戳

仅仅基于用户行为数据设计的推荐算法成为协同过滤算法，具体细分为基于邻域的方法、隐语义模型、基于图的随机游走算法。目前业界应用较为广泛的方法是基于邻域的方法，基于邻域的方法包括下面两种算法：

- 基于用户的协同过滤算法

- 基于物品的协同过滤算法

## 基于用户的协同过滤算法

1. 找到和目标用户兴趣相似的用户集合

2. 将兴趣相似用户喜欢且目标用户不曾见过的物品推荐给目标用户

In [1]:
import math
def UserSimilarity(train):
    res = {}
    for u in train:
        res.setdefault(u,0)
        for v in train:
            if u != v:
                res[u].setdefault(v,[])
                res[u][v] = len(train[u] & train[v])
                res[u][v] /= math.sqrt( len(train[u]) * len(train[v]))
    return res

In [2]:
from collections import defaultdict
def UserSimilarity(train):
    res = defaultdict(dict)
    for u in train:
        for v in train:
            if u != v:
                # 以余弦相似度为计算核心
                res[u][v] = len(train[u] & train[v])
                res[u][v] /= math.sqrt( len(train[u]) * len(train[v]))
    return res

In [3]:
train_data = {'A':['a','b','c'],'B':['a','c']} # 会报错
train_data = {'A':{'a','b','c'},'B':{'a','c'}} # 只能用set结构，用set结构也比较合理
UserSimilarity(train_data)

defaultdict(dict,
            {'A': {'B': 0.8164965809277261}, 'B': {'A': 0.8164965809277261}})

### 看升级！！！

In [14]:
def UserSimilarity(train):
    item_user = defaultdict(set)
    # item_user = defaultdict(list)
    # item_user = defaultdict(dict)
    for user, items in train.items():
        for item in items:
            # item_user[item].append(user)
            item_user[item].add(user)
    print('用户-物品转置表：',item_user)
    co_num = defaultdict(dict)
    # user_num = {}
    user_num = defaultdict(int)
    for item, users in item_user.items():
        for u in users:
            user_num.setdefault(user,0)
            user_num[u] += 1
            for v in users:
                co_num[u].setdefault(v,0)
                if u != v:
                    co_num[u][v] += 1
    print('单个用户评价的物品个数', user_num)        
    print('协同用户评价的物品个数', co_num)
    
    res = defaultdict(dict)
    for user, co_user in co_num.items():
        for u,n in co_user.items():
            res[user][u] = n
            res[user][u] /= math.sqrt(user_num[user] * user_num[u])
        
    print(res)
UserSimilarity(train_data)

用户-物品转置表： defaultdict(<class 'set'>, {'b': {'A'}, 'a': {'A', 'B'}, 'c': {'A', 'B'}})
单个用户评价的物品个数 defaultdict(<class 'int'>, {'B': 2, 'A': 3})
协同用户评价的物品个数 defaultdict(<class 'dict'>, {'A': {'A': 0, 'B': 2}, 'B': {'A': 2, 'B': 0}})
defaultdict(<class 'dict'>, {'A': {'A': 0.0, 'B': 0.8164965809277261}, 'B': {'A': 0.8164965809277261, 'B': 0.0}})
